# transformers fine tune for model: bert-base-cased dataset: yelp_review_full


In [2]:
from datasets import load_dataset
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    pipeline
)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

import torch
from datetime import datetime

# output dir prefix
ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")
output_dir_base = f"./finetune/{ts}"
model_dir = f"{output_dir_base}/models/yelp_bert_finetuned"
logging_dir = f"{output_dir_base}/logs"
results_dir = f"{output_dir_base}/results"
use_full_dataset = False  # Set to False for quick testing with smaller subset

# =============================================
# 1. LOAD DATASET AND MODELS
# =============================================
dataset = load_dataset("yelp_review_full")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

# Load BOTH models:
# - base_model: original BERT (no fine-tuning)
# - model: will be fine-tuned
base_model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

# =============================================
# 2. PREPROCESS DATA
# =============================================
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# check if we are using the full dataset or a smaller subset
print(f"Dataset size: {len(encoded_dataset['train'])} training examples, {len(encoded_dataset['test'])} test examples")
if use_full_dataset:
    print("Using full dataset for training and evaluation.")
    train_dataset = encoded_dataset["train"]
    eval_dataset = encoded_dataset["test"]
else:
    print("Using smaller subset of 1000 examples for quick testing.")
    train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(1000))
    eval_dataset = encoded_dataset["test"].shuffle(seed=42).select(range(1000))

# Use the official training split from the dataset
#full_train = encoded_dataset["train"]

# Use Hugging Face's built-in train_test_split() method
#split_data = full_train.train_test_split(test_size=0.1, seed=42)

# Extract the two parts
#train_data = split_data["train"]  # 90% of training data
#val_data   = split_data["test"]   # 10% of training data

# Now take smaller subsets (e.g. 1000 examples each) for quick testing
#train_dataset = train_data.shuffle(seed=42).select(range(1000))
#eval_dataset  = val_data.shuffle(seed=42).select(range(1000))

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# =============================================
# 3. METRICS FUNCTION
# =============================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }

# =============================================
# 4. TRAINING CONFIGURATION
# =============================================
training_args = TrainingArguments(
    output_dir=results_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir=logging_dir,
    logging_steps=1,
    logging_strategy="steps"
)

# =============================================
# 5. BEFORE FINE-TUNING: BASELINE INFERENCE
# =============================================
print("=== BEFORE FINE-TUNING (BASE BERT) INFERENCE ===")

# Create a pipeline with the base (untrained) model
base_classifier = pipeline(
    "text-classification",
    model=base_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Sample reviews (1-star to 5-star)
sample_reviews = [
    "Terrible service, worst food ever. Would not recommend.",  # Likely 1-star
    "It was okay. Nothing special but not bad either.",          # Likely 3-star
    "Absolutely amazing experience. Food was perfect!",          # Likely 5-star
    "The staff was rude and the place was dirty.",               # Likely 1-2 star
    "Excellent quality, friendly employees, will come back!"     # Likely 4-5 star
]

print("Sample Reviews & Base Model Predictions:")
for review in sample_reviews:
    result = base_classifier(review)
    label = int(result[0]['label'].split('_')[1])  # Extract number from 'LABEL_0' etc.
    print(f"Review: {review[:50]}... | Predicted: {label}-star")

# =============================================
# 6. FINE-TUNE THE MODEL
# =============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("=== STARTING FINE-TUNING ===")
trainer.train()
eval_results = trainer.evaluate()
print(f"Final evaluation: {eval_results}")

# =============================================
# 7. AFTER FINE-TUNING: CUSTOM MODEL INFERENCE
# =============================================
print("=== AFTER FINE-TUNING (CUSTOM BERT) INFERENCE ===")

# Save and reload the fine-tuned model to ensure clean inference
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# Create pipeline with the fine-tuned model
fine_tuned_classifier = pipeline(
    "text-classification",
    model=model_dir,
    tokenizer=model_dir,
    device=0 if torch.cuda.is_available() else -1
)

print("Sample Reviews & Fine-Tuned Model Predictions:")
for review in sample_reviews:
    result = fine_tuned_classifier(review)
    label = int(result[0]['label'].split('_')[1])  # Extract number from 'LABEL_0' etc.
    print(f"Review: {review[:50]}... | Predicted: {label}-star")

# =============================================
# 8. OPTIONAL: Compare a specific review side-by-side
# =============================================
if len(sample_reviews) > 0:
    test_review = sample_reviews[0]  # First sample review
    print(f"=== SIDE-BY-SIDE COMPARISON FOR REVIEW ===")
    print(f"Review Text: {test_review}")

    base_result = base_classifier(test_review)
    base_label = int(base_result[0]['label'].split('_')[1])
    base_score = base_result[0]['score']
    print(f"[BASE BERT]   Predicted: {base_label}-star (confidence: {base_score:.2f})")

    fine_tuned_result = fine_tuned_classifier(test_review)
    fine_label = int(fine_tuned_result[0]['label'].split('_')[1])
    fine_score = fine_tuned_result[0]['score']
    print(f"[FINE-TUNED] Predicted: {fine_label}-star (confidence: {fine_score:.2f})")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Dataset size: 650000 training examples, 50000 test examples
Using smaller subset of 1000 examples for quick testing.
=== BEFORE FINE-TUNING (BASE BERT) INFERENCE ===
Sample Reviews & Base Model Predictions:


C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Review: Terrible service, worst food ever. Would not recom... | Predicted: 4-star
Review: It was okay. Nothing special but not bad either.... | Predicted: 0-star
Review: Absolutely amazing experience. Food was perfect!... | Predicted: 4-star
Review: The staff was rude and the place was dirty.... | Predicted: 4-star
Review: Excellent quality, friendly employees, will come b... | Predicted: 0-star


C:\Users\bill\AppData\Local\Temp\ipykernel_19896\3425049595.py:132: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.417800,1.377739,0.457000,0.436432
2,0.922300,1.148504,0.519000,0.496160
3,0.843600,1.107333,0.531000,0.517961


C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\dev\inst\miniconda3\env

Final evaluation: {'eval_loss': 1.1073333024978638, 'eval_accuracy': 0.531, 'eval_f1': 0.517960864730635, 'eval_runtime': 146.9064, 'eval_samples_per_second': 6.807, 'eval_steps_per_second': 0.429, 'epoch': 3.0}
=== AFTER FINE-TUNING (CUSTOM BERT) INFERENCE ===


Device set to use cpu
C:\dev\inst\miniconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Sample Reviews & Fine-Tuned Model Predictions:
Review: Terrible service, worst food ever. Would not recom... | Predicted: 0-star
Review: It was okay. Nothing special but not bad either.... | Predicted: 1-star
Review: Absolutely amazing experience. Food was perfect!... | Predicted: 4-star
Review: The staff was rude and the place was dirty.... | Predicted: 0-star
Review: Excellent quality, friendly employees, will come b... | Predicted: 4-star
=== SIDE-BY-SIDE COMPARISON FOR REVIEW ===
Review Text: Terrible service, worst food ever. Would not recommend.
[BASE BERT]   Predicted: 4-star (confidence: 0.22)
[FINE-TUNED] Predicted: 0-star (confidence: 0.76)
